<a href="https://colab.research.google.com/github/Icrz11/CS114.K21/blob/master/Sacrasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Thành viên thực hiện

#Nguyễn Hữu Khang - 18520892
#Hồ Đăng Tuệ - 18521611


#Giới thiệu về Bài toán nhận biết Sarcasm
  * Trong thực tế có rất nhiều title báo giật tít trên các trang báo lá cải, cũng như mạng xã hội

  * Vì vậy mà việc detect các title này để xác minh tin nào là tin chính thống, tin nào là tin châm biếm đang là vấn đề lớn (phát triển rộng hơn nữa là bài toán về kiểm tra fake news)

  * Dataset được sử dụng trong bài toán này có sẵn ở kaggle, và được thu thập từ 2 nguồn đó là trang web https://www.huffpost.com/ - chuyên tin chính thống, cùng với  https://www.theonion.com/ - Chuyên tin lá cải


#Cách thu thâp Dataset:

> Sử dụng tool có tên Data Miner (công cụ có sẵn ở google chrome)
Link của tool: https://data-miner.io/

Sao khi mở tool lên, ta tạo 1 receipt mới
1. Start
2. Rows - Các cột trong file mà ta sẽ xuất ra ở đây ta cần thu thập headline, cho nên headline sẽ là 1 cột trong Rows
3. Cols - Nếu ta cần thêm thông tin nào nữa thì chỉ cần add thêm column vào và quay lại tab Rows để bổ sung nội dung hiển thị trong column đó
4. Nav - Đại loại là ta sẽ thu thập tiếp tục dữ liệu ở 1 trang kế tiếp và gắn dữ liệu đó vào đuôi của dữ liệu đã crawl trước đó
5. Actions - ta sẽ thực hiện 1 số hành động như kéo tới cuối trang, click vào 1 button nào đó
6. JS - để add Script vào
7. Save - để save là receipt của ta

Em sử dụng tab Rows + Nav + Save để hoàn thành receipt crawl dữ liệu của mình

> Rows em sẽ chọn nội dung là các headline mình cần crawl

> Nav em sẽ set nút button để chuyển 

> Cuối cùng là save lại Receipt và chạy tự động để crawl dữ liệu về

# Lấy Data từ Kaggle về

In [5]:
!wget https://raw.githubusercontent.com/Icrz11/CS114.K21/master/Sarcasm_Headlines_Dataset.json

--2020-06-26 12:25:20--  https://raw.githubusercontent.com/Icrz11/CS114.K21/master/Sarcasm_Headlines_Dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5616830 (5.4M) [text/plain]
Saving to: ‘Sarcasm_Headlines_Dataset.json’

Sarcasm_Headlines_D 100%[===================>]   5.36M  14.6MB/s    in 0.4s    

2020-06-26 12:25:20 (14.6 MB/s) - ‘Sarcasm_Headlines_Dataset.json’ saved [5616830/5616830]



Download file Sarcasm Headline từ github (được em lưu sẵn)

In [6]:
#read data
import pandas as pd
df = pd.read_json("/content/Sarcasm_Headlines_Dataset.json", lines = True)
df.head()
df = df.drop(['article_link'], axis = 1)
print (df)

                                                headline  is_sarcastic
0      former versace store clerk sues over secret 'b...             0
1      the 'roseanne' revival catches up to our thorn...             0
2      mom starting to fear son's web series closest ...             1
3      boehner just wants wife to listen, not come up...             1
4      j.k. rowling wishes snape happy birthday in th...             0
...                                                  ...           ...
26704               american politics in moral free-fall             0
26705                            america's best 20 hikes             0
26706                              reparations and obama             0
26707  israeli ban targeting boycott supporters raise...             0
26708                  gourmet gifts for the foodie 2014             0

[26709 rows x 2 columns]


> Dùng pandas đọc file .json 

> Kế đến ta sẽ drop cột article link đi (vì cột này không cần thiết lắm trong bài toán của ta)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26709 entries, 0 to 26708
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   headline      26709 non-null  object
 1   is_sarcastic  26709 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 417.5+ KB


1 số info từ Data ta vừa thu thâp được, trong đó data này có 26709 mẫu bao gồm 2 thông tin là headlibe + nhãn có phải là tin châm biếm hay không

In [52]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors
import string 
from string import digits
from string import punctuation
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
for i in df['headline']:
  i = str(i)

Import tất các thư viện cần dùng trong bài toán này

1. Numpy - Xử lí các ma trận
2. Sklearn - Các hàm liên quan đến thuật toán máy học cũng như đánh giá model
3. String - thư viện xử lí các kí tự 

# Tiền xử lí dữ liệu

In [10]:
hl_clean = []
for i in df['headline']:
  clean = i.translate(str.maketrans('', '', punctuation))
  clean = clean.translate(str.maketrans('', '', digits))
  hl_clean.append(clean)

print('Before: ')
print(df['headline'][37])
print ('After: ')
print (hl_clean[37])
for i in hl_clean:
  i = i.lower()


Before: 
'moana' sails straight to the top of the box office with massive $81.1 million opening
After: 
moana sails straight to the top of the box office with massive  million opening


Loại bỏ các thành phần không đóng vai trò quan trọng trong câu của chúng ta

Các thành phần bị loại bỏ:
> Các chữ số 

> Các dấu câu

In [11]:
hl_tokens = []
for i in hl_clean:
  i = i.split()
  hl_tokens.append(i)

print ('before: ')
print (hl_clean[100])
print ('after: ')
print (hl_tokens[100])

before: 
demi lovato drops emotional nightingale music vid
after: 
['demi', 'lovato', 'drops', 'emotional', 'nightingale', 'music', 'vid']


Tách các từ trong câu ra thành các token riêng lẽ ở trên là ví dụ

In [12]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
Lemmantizer = WordNetLemmatizer()
hl_lemma = []
count = 0
print (hl_tokens[10])
for i in (hl_tokens):
  hl_lemma.append([])
  for j in i:
    hl_lemma[count].append(Lemmantizer.lemmatize(j))
  count+=1
print (hl_lemma[10])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
['airline', 'passengers', 'tackle', 'man', 'who', 'rushes', 'cockpit', 'in', 'bomb', 'threat']
['airline', 'passenger', 'tackle', 'man', 'who', 'rush', 'cockpit', 'in', 'bomb', 'threat']


Chuyển toàn bộ từ về form cơ bản của nó

Ví dụ: goes -> go


In [133]:
import nltk
nltk.download ('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words = list(stop_words)
hl_re = []
count = 0
for i in hl_lemma:
  hl_re.append([])
  for j in i:
    if (j not in stop_words):
      hl_re[count].append(j)
  count+=1



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
Lemmantizer = WordNetLemmatizer()
def Preprocessing_Data (Headline):
  Headline = Headline.lower()
  #Dùng để loại bỏ các dấu câu, chữ số
  clean = Headline.translate(str.maketrans('', '', punctuation))
  clean = clean.translate(str.maketrans('', '', digits))
  #Tách các từ ra thành các token
  clean = clean.split()
  #Chuyển chúng về dạng "root" của chúng
  result = []
  for i in clean:
    result.append(Lemmantizer.lemmatize(i))
  return result

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Đưa dữ liệu vào model để Trainning (sử dụng bag of words để features extraction)

In [15]:
tmp = hl_re
wordfreq = {}
for i in tmp:
  for token in i:
    if token not in wordfreq.keys():
      wordfreq[token] = 1
    else:
      wordfreq[token] += 1


In [16]:
sort_wordfreq = sorted(wordfreq.items(), key=lambda x: x[1], reverse=True)


In [17]:
vector_len = 2000
tmp2 = []
for i in range (vector_len):
  tmp2.append(sort_wordfreq[i][0])

In [18]:
sentence_vectors = []
for sentence in hl_re:
  sent_vec = np.zeros(shape = (vector_len,))
  for i,token in enumerate(tmp2):
    if token in sentence:
      sent_vec[i] = 1
  sentence_vectors.append(sent_vec)

In [64]:
def Bag_Of_Word(Headline):
  sent_vec = np.zeros(shape = (vector_len,))
  for i in range (len(Headline)):
    if (Headline[i] in tmp2):
      sent_vec[i] = 1
    return sent_vec

In [32]:
Y = df.iloc[:,1]
X = sentence_vectors
X = np.array (X)
Y = np.array(Y)
print (len(X))
print (len(Y))

26709
26709


In [33]:
X_train, X_test, y_train, y_test = train_test_split (X,Y, test_size = 0.2, random_state = 30)


### Naive Bayes

In [24]:
from sklearn.naive_bayes import GaussianNB
model1 = GaussianNB()
model1.fit (X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [25]:
re1 = model1.predict (X_test)

In [128]:
confusion_matrix(y_test, re1).ravel()

array([1817, 1190,  426, 1909])

Từ confusion matrix ta thấy được số nhãn được:

1817 -> Số nhãn được dự đoán đúng thuộc loại negative (TN) | 1190 -> Số nhãn được dự đoán sai loại positive (FP)

426 -> Số nhãn được dự đoán sai loại Negative | 1909 -> Số nhãn được dự đoán đúng thuộc loại positive

Với negative = 0 và positive = 1 (0 là non sacrasm và 1 là sacrasm)

In [59]:
f1_model1 = f1_score(y_test, re1)
print ("F1_score: ", f1_model1)
print ("Accurancy: ",accuracy_score(y_test, re1))

F1_score:  0.7026131762973868
Accurancy:  0.6974915761886934


Ta thấy chỉ số F1-score ở model này khá cao (0.70) với F1 score được tính bởi 

$ (2*Precision*Recall) 
/ (Precision + Recall) $

Cho nên nếu cả precision và recall đều cao thì F1 - score mới cao được


      > $Precision = 1909 / (1909 + 426) = 0.817$

      > $Recall = 1909 / (1909 + 1190) = 0.61$

>Precision biểu thị cho việc tiên đoán số nhãn true positive tronng số những điểm được phân loại là positive 

>Recall biểu thị việc tiên đoán số nhãn True trong những điểm thật sự là positive

### K-NN


In [34]:
from sklearn.neighbors import KNeighborsClassifier
model2 = KNeighborsClassifier()
model2.fit (X_train, y_train)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [36]:
re2 = model2.predict (X_test)

In [129]:
confusion_matrix(y_test, re2).ravel()

array([2032,  975,  998, 1337])

2032 -> Số nhãn được dự đoán đúng thuộc loại negative (TN) | 975 -> Số nhãn được dự đoán sai loại positive (FP)

998 -> Số nhãn được dự đoán sai loại Negative | 1337 -> Số nhãn được dự đoán đúng thuộc loại positive

In [58]:
f1_model2 = f1_score(y_test, re2)
print ("F1_score: ", f1_model2)
print ("Accurancy: ",accuracy_score(y_test, re2))

F1_score:  0.5754250053798149
Accurancy:  0.6306626731561213


 F1 - score ở model KNN là 0.57 và Accurancy là 0.63, cả 2 đều khá thấp

### SVM

In [43]:
from sklearn import svm
model3 = svm.SVC()
model3.fit (X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [44]:
re3 = model3.predict (X_test)

In [131]:
confusion_matrix(y_test, re3).ravel()

array([2593,  414,  791, 1544])

2593 -> Số nhãn được dự đoán đúng thuộc loại negative (TN) | 414 -> Số nhãn được dự đoán sai loại positive (FP)

791 -> Số nhãn được dự đoán sai loại Negative | 1554 -> Số nhãn được dự đoán đúng thuộc loại positive

In [57]:
f1_model3 = f1_score(y_test, re3)
print ("F1_score: ", f1_model3)
print ("Accurancy: ",accuracy_score(y_test, re3))

F1_score:  0.7193105054740274
Accurancy:  0.7744290527892175


 F1 - score ở model SVM là 0.71 và Accurancy là 0.77 Đây là model ổn nhất

### Lưu và Load model


In [121]:
import joblib

In [122]:
joblib.dump(model1, 'model_sac_naivebayes.sav')
joblib.dump(model2, 'model_sac_KNN.sav')
joblib.dump(model3, 'model_sac_SVM.sav')

['model_sac_SVM.sav']

In [ ]:
model1 = joblib.load('')
model2 = joblib.load('')
model3 = joblib.load('')

:v em lưu lại model nhưng kh post lại trên github được do dung lượng file khá nặng, thầy chạy lại từ đầu giúp em <3

# Đưa dữ liệu gồm 2k headline được crawl về để test model



In [21]:
!wget https://raw.githubusercontent.com/Icrz11/CS114.K21/master/test_data.csv

--2020-06-26 12:30:11--  https://raw.githubusercontent.com/Icrz11/CS114.K21/master/test_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 165600 (162K) [text/plain]
Saving to: ‘test_data.csv’

test_data.csv       100%[===================>] 161.72K  --.-KB/s    in 0.04s   

2020-06-26 12:30:12 (3.73 MB/s) - ‘test_data.csv’ saved [165600/165600]



In [60]:
test = pd.read_csv ('/content/test_data.csv')
test.head()

,headline,is_sarcastic
0,BirthdayForBreonna Marks What Would've Been Br...,0
1,TheShowMustBePaused Was Eclipsed By #BlackOutT...,0
2,"Again, Again, Again!' Exclaims Clapping, Grinn...",1
3,"And Then There Were 23,' Says Wayne Messam Cro...",1
4,And Then Those 12 People Send It To 12 People ...,1


In [61]:
Y = test['is_sarcastic']
Y = np.array(Y)
X = test['headline']

In [62]:
X.head()


0    BirthdayForBreonna Marks What Would've Been Br...
1    TheShowMustBePaused Was Eclipsed By #BlackOutT...
2    Again, Again, Again!' Exclaims Clapping, Grinn...
3    And Then There Were 23,' Says Wayne Messam Cro...
4    And Then Those 12 People Send It To 12 People ...
Name: headline, dtype: object

In [65]:
X_train = []
for i in X:
  X_train.append(Preprocessing_Data(i))
kHeadline = []
for i in X_train:
  kHeadline.append(Bag_Of_Word(i))

In [66]:
re_test1 = model1.predict (kHeadline)
re_test2 = model2.predict (kHeadline)
re_test3 = model3.predict (kHeadline)

In [132]:
print ("confusion matrix cua Naive Bayes:\n",confusion_matrix(Y, re_test1).ravel())
print ("confusion matrix cua KNN:\n",confusion_matrix(Y, re_test2).ravel())
print ("confusion matrix cua SVM:\n",confusion_matrix(Y, re_test3).ravel())


confusion matrix cua Naive Bayes:
 [   0 1156    0 1003]
confusion matrix cua KNN:
 [580 576 524 479]
confusion matrix cua SVM:
 [1156    0 1003    0]


In [117]:
print ('Naive Bayes:\n', classification_report(Y, re_test1))
print ('\n')
print ('accurancy cua Naive Bayes: ',accuracy_score(Y, re_test1))

Naive Bayes:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00      1156
           1       0.46      1.00      0.63      1003

    accuracy                           0.46      2159
   macro avg       0.23      0.50      0.32      2159
weighted avg       0.22      0.46      0.29      2159



accurancy cua Naive Bayes:  0.4645669291338583


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Ta thấy model khi test trên tập dữ liệu 2k headline mới thì chỉ số recall cao ngất ngưỡng (1.0) có nghĩa là trong 2k headline mới thì khả năng model phát hiện 1 tin có phải sacrasm là 100%, tuy nhiên precision của model khá thấp (0.46) có nghĩa là model có 46% đúng khi khẳng định 1 tin là sacrasm (Model tiên đoán khá tệ). 

=> Model dự đoán tất cả headline đầu vào là sacrasm.

In [119]:
print ('KNN:\n', classification_report(Y, re_test2))
print ('\n')
print ('accurancy cua KNN: ',accuracy_score(Y, re_test2))

KNN:
               precision    recall  f1-score   support

           0       0.53      0.50      0.51      1156
           1       0.45      0.48      0.47      1003

    accuracy                           0.49      2159
   macro avg       0.49      0.49      0.49      2159
weighted avg       0.49      0.49      0.49      2159



accurancy cua KNN:  0.4905048633626679


Tuy chỉ số recall cũng như precision của model KNN không cao, nhưng có thể chấp nhận được hơn là model Naive Bayes ở trên

=> Model dự đoán tất cả headline đầu vào là sacrasm.

In [120]:
print ('SVM:\n',classification_report(Y, re_test3))
print ('\n')
print ('accurancy cua SVM : ',accuracy_score(Y, re_test3))

SVM:
               precision    recall  f1-score   support

           0       0.54      1.00      0.70      1156
           1       0.00      0.00      0.00      1003

    accuracy                           0.54      2159
   macro avg       0.27      0.50      0.35      2159
weighted avg       0.29      0.54      0.37      2159



accurancy cua SVM :  0.5354330708661418


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Tương tự như model Naive Bayes, model cũng dự đoán sai hoàn toàn

Lần này model dự đoán tất cả headline đầu vào đều không phải là sacrasm

# Chương trình đưa vào 1 Headline và dự đoán

In [127]:
#Bag of word
Inp = input ('Moi nhap headline: ')
Inp = Preprocessing_Data(Inp)
Inp = Bag_Of_Word(Inp)
Inp = np.array(Inp)
Inp = Inp.reshape(1, -1)
Kqua1 = model1.predict(Inp)
Kqua2 = model2.predict(Inp)
Kqua3 = model3.predict(Inp)
def check(Kqua):
  if (Kqua == 1):
    re = 'Day la tin Sacrasm'
  else:
    re = 'Day khong phai tin Sacrasm'
  return re

print ('Model Naive Bayes: ', check(Kqua1))
print ('Model KNN: ', check(Kqua2))
print ('Model SVM: ', check(Kqua3))

Moi nhap headline: I Am George Floyd': Air Force's Top Enlisted Leader Denounces Police Killing
Model Naive Bayes:  Day la tin Sacrasm
Model KNN:  Day la tin Sacrasm
Model SVM:  Day khong phai tin Sacrasm


# Trả lời câu hỏi


1. Mô tả bài toán ở trên cùng

2. Thu thâp headline ở 2 trang web trên, em sử dụng 1 tool extentions ở gooogle chrome có tên là Data miner để crawl dữ liệu

3. Chúng em sử dụng 2 phương pháp để rút trích đặc trưng là Bag of Words (túi từ - sử dụng khoảng 2000 từ thường xuất hiện nhất)

4. Em sử test trên 3 thuật toán là KNN, Naive Bayes, svm và thấy model khi dùng svm là cho tỉ lệ dự đoán cao nhất khoảng 77% trên tập test đc chia ra từ Data. Em chưa tuning bất kì thuật toán nào cả.

5. Model đã train sẽ được lưu lại thông qua thư viện joblib, chương trình đưa 1 headline mới vào để dự đoán được viết ở phía trên.


6. Performance ở trường hợp 3 trường hợp của 3 model Naive Bayes, KNN, SVM đều khá tệ

  1. Naive Bayes -> Dự đoán tất cả headline đầu vào đều là Sacrasm
  2. KNN -> Dự đoán chỉ số F1 Score cũng như Accurancy khá thấp
  3. SVM -> Dự đoán toàn bộ headline đầu vào đều là non-sacrasm

  Nguyên do dự đoán khá tệ:
  1. Có thể chưa tuning hyperparmeters
  2. Do khâu rút trích đặc trưng bằng Bag of words khá tệ (đây là kĩ thuật thô sơ, không biểu diễn không rõ giá trị của các từ trong câu như các kĩ thuật tiên tiến hơn, chỉ biểu diễn được 1 câu tốt nhất khi tất cả từ trong câu đó đều có trong từ vựng (được tạo ra trước đó trong túi từ), và nếu là phương pháp BOW thì câu "Tôi đi học" sẽ tương đương với "đi học tôi" hoặc "đi tôi học".

  Có thể em sẽ nghiên cứu thêm những kĩ thuật rút trích đặc trưng hiện đại hơn như word embedding, word2vec để cải thiện hiệu suất của model.


